# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so don't need to load anything from disk.

In [ ]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 3
DATASET_SIZE = 200000
ds = EchoDataset(delay=3, size=DATASET_SIZE)
a=ds.__iter__()
#print(next(a))
#print(idx_to_onehot(torch.tensor(1),5))
l=15
delays=torch.tensor(5)
delay1=torch.full(size=(l,1),fill_value=delays-1)
delay1=torch.squeeze(delay1)
print(delay1)
delay1=idx_to_onehot(delay1,8)
print(delay1)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
tensor([[0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.]])


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [ ]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size=hidden_size
    self.rnn=torch.nn.GRU(
        input_size=27,
        hidden_size=hidden_size
    )
    self.linear=torch.nn.Linear(in_features=hidden_size,out_features=27)
    self.maxx=torch.nn.Softmax()
  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    x,hidden=self.rnn(x)
    x=self.linear(x)
    return x
    pass

  @torch.no_grad()
  def test_run(self, s,device):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.
    # TODO
    te=[]
    for c in s:
      if c!=' ':
        te.append(ord(c)-96)
      else:
        te.append(0)
    te=idx_to_onehot(torch.tensor(te))
    te=te.to(device)
    output=self.forward(te)
    out,ind=torch.max(output,1)
    string=[]
    #print(output)
    for i in ind:
      if i != 0:
        string.append(chr(i+96))
        #print('yesss')
      else:
        string.append(' ')
    return string
    pass

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [ ]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  D=3
  if torch.cuda.is_available():
    device = torch.device("cuda:0")
  else:
    device = torch.device("cpu")
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s,device)
    print(s,result)
    assert D > 0, 's[:-D] won\'t work for D=0'
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [ ]:
import time
start_time = time.time()

# TODO: initialize and train your model here.
model = GRUMemory(hidden_size=1000)
criterion = torch.nn.BCEWithLogitsLoss() # TODO
optimizer = torch.optim.RMSprop(model.parameters(),lr=0.005)
model.train()
if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")
NN=8000
for i in range(NN):
  input,target=next(a)
  #print(input)
  input=idx_to_onehot(input)
  target=idx_to_onehot(target)
  input=input.to(device)
  target=target.to(device)
  output=model(input)
  #print(output)
  loss=criterion(output,target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
print(accuracy)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

zvclahsvueeizwv [' ', ' ', ' ', 'z', 'v', 'c', 'l', 'a', 'h', 's', 'v', 'u', 'e', 'e', 'i']
ubnrabspvknnbepwpvbtt [' ', ' ', ' ', 'u', 'b', 'n', 'r', 'a', 'b', 's', 'g', 'v', 'k', 'n', 'n', 'b', 'e', 'p', 'w', 'p', 'v']
uniakinieskmpgmsqswxgve [' ', ' ', ' ', 'u', 'n', 'i', 'a', 'k', 'i', 'n', 'i', 'e', 's', 'k', 'm', 'p', 'g', 'm', 's', 'q', 's', 'w', 'x']
ctfmozyumzxcfsyzjgs [' ', ' ', ' ', 'c', 't', 'f', 'm', 'o', 'z', 'y', 'u', 'm', 'z', 'x', 'c', 'f', 's', 'y', 'z']
fzewnncuqxtbjgfl [' ', ' ', ' ', 'f', 'z', 'e', 'w', 'n', 'n', 'c', 'u', 'q', 'x', 't', 'b', 'j']
ocwmaulferxvjrqlcqskya [' ', ' ', ' ', 'o', 'c', 'w', 'm', 'a', 'u', 'l', 'f', 'e', 'r', 'x', 'v', 'j', 'r', 'q', 'l', 'c', 'q', 's']
rpvwwbobbfmcmumosfdwc [' ', ' ', ' ', 'r', 'p', 'v', 'w', 'w', 'b', 'b', 'b', 'b', 'f', 'm', 'c', 'm', 'u', 'm', 'o', 's', 'f']
biluzitapirsmfxytlplhrb [' ', ' ', ' ', 'b', 'i', 'l', 'u', 'z', 'i', 't', 'a', 'p', 'i', 'r', 's', 'm', 'f', 'x', 'y', 't', 'l', 'p', 'l']
wdhgglfucsldhqdf [' ', '

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay stays constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [ ]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=100000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result
  
ds1=VariableDelayEchoDataset()
a1=ds1.__iter__()


### Model

And the model.

In [ ]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay,device):
    super().__init__()
    #TODO
    self.hidden_size=hidden_size
    self.rnn=torch.nn.GRU(
        input_size=27+9,
        hidden_size=hidden_size
    )
    self.linear=torch.nn.Linear(in_features=hidden_size,out_features=27)
    self.maxx=torch.nn.Softmax()
    self.device=device
    self.max_delay=max_delay
  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    l,c=x.size()
    delay1=torch.full(size=(l,1),fill_value=delays)
    delay1=torch.squeeze(delay1)
    #print(delay1)
    delay1=idx_to_onehot(delay1,9)
    delay1=delay1.to(device)
    x=torch.cat((x,delay1),1)
    #print(x)
    x=x.to(device)
    x,hidden=self.rnn(x)
    x=self.linear(x)
    return x
    pass

  @torch.no_grad()
  def test_run(self, s, delay,device):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    te=[]
    for c in s:
      if c!=' ':
        te.append(ord(c)-96)
      else:
        te.append(0)
    te=idx_to_onehot(torch.tensor(te))
    te=te.to(device)
    output=self.forward(te,delay)
    out,ind=torch.max(output,1)
    string=[]
    #print(output)
    for i in ind:
      if i != 0:
        string.append(chr(i+96))
        #print('yesss')
      else:
        string.append(' ')
    return string
    pass


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [ ]:
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d,device=device)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [ ]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20
if torch.cuda.is_available():
  device = torch.device("cuda:0")
else:
  device = torch.device("cpu")

# TODO: implement model training here.
model = VariableDelayGRUMemory(2000,8,device)
criterion = torch.nn.BCEWithLogitsLoss() # TODO
optimizer = torch.optim.RMSprop(model.parameters(),lr=0.0003)
model.train()
if torch.cuda.is_available():
  model = model.cuda()
  criterion = criterion.cuda()
NN=26000
for i in range(NN):
  input,delay,target=next(a1)
  #print(input)
  input=idx_to_onehot(input)
  target=idx_to_onehot(target)
  input=input.to(device)
  target=target.to(device)
  delay=torch.tensor(delay)
  delay=delay.to(device)
  output=model(input,delay)
  #print(output)
  loss=criterion(output,target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if i%100==0:
    print(i)

end_time = time.time()
print(test_variable_delay_model(model) )
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18